In [3]:
# For Google Colaboratory
import sys, os
if 'google.colab' in sys.modules:
    # mount google drive
    from google.colab import drive
    drive.mount('/content/gdrive')
    # find automatically the path of the folder containing "file_name" :
    file_name = 'mlp.ipynb'
    import subprocess
    path_to_file = subprocess.check_output('find . -type f -name ' + str(file_name), shell=True).decode("utf-8")
    path_to_file = path_to_file.replace(file_name,"").replace('\n',"")
    # if previous search failed or too long, comment the previous line and simply write down manually the path below :
    #path_to_file = '/content/gdrive/My Drive/CS5242_2021_codes/codes/labs_lecture06/lab01_mnist_multilayer'
    print(path_to_file)
    # change current path to the folder containing "file_name"
    os.chdir(path_to_file)
    !pwd

Mounted at /content/gdrive
./gdrive/MyDrive/MyProject/
/content/gdrive/MyDrive/MyProject


In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from random import randint
import time

In [5]:
device= torch.device("cuda")
# device= torch.device("cpu")
print(device)

cuda


#### Define dataset

In [6]:
def get_error( scores , labels ):

    bs=scores.size(0)
    predicted_labels = scores.argmax(dim=1)
    indicator = (predicted_labels == labels)
    num_matches=indicator.sum()
    
    return 1-num_matches.float()/bs  

In [45]:
# data_path = './Data/Clean/'
# data_path = './Data/Clean_32x32/'
# data_path = './Data/CleanerV1/'
# data_path = './Data/CleanerV1_32x32/'
# data_path = './Data/CleanerV2/'
data_path = './Data/CleanerV2_32x32/'
# data_path = './Data/Kaggle/'
# data_path = './Data/Kaggle_32x32/'

# Color
train_data=torch.load(data_path+'training_data_color.pt').squeeze().to(device)
train_label=torch.load(data_path+'training_labels_color.pt').squeeze().to(device)
test_data=torch.load(data_path+'testing_data_color.pt').squeeze().to(device)
test_label=torch.load(data_path+'testing_labels_color.pt').squeeze().to(device)

# B&W Data
train_data=torch.load(data_path+'training_data_bw.pt').squeeze().to(device)
train_label=torch.load(data_path+'training_labels_bw.pt').squeeze().to(device)
test_data=torch.load(data_path+'testing_data_bw.pt').squeeze().to(device)
test_label=torch.load(data_path+'testing_labels_bw.pt').squeeze().to(device)

RuntimeError: ignored

In [8]:
print(train_data.size(), train_label.size())
print(test_data.size(), test_label.size())
train_size = train_data.size()[0]
test_size = test_data.size()[0]
print("Training data size is: ", train_size)
print("Testing data size is: ", test_size)

torch.Size([3381, 28, 28]) torch.Size([3381])
torch.Size([1461, 28, 28]) torch.Size([1461])
Training data size is:  3381
Testing data size is:  1461


In [9]:
# 784 for input image of size 28 * 28
# 2 for output classes 0 and 1
image_size = 784

### One Layer MLP

In [ ]:
class one_layer_net(nn.Module):
    def __init__(self, input_size, output_size):
        super(one_layer_net , self).__init__()
        self.linear_layer = nn.Linear(input_size, output_size, bias=True)

    def forward(self, x):
        scores = self.linear_layer(x)
        return scores

In [ ]:
net_one = one_layer_net(image_size, 2)
print(net_one)

one_layer_net(
  (linear_layer): Linear(in_features=784, out_features=2, bias=True)
)


#### Send the weights of the networks to the GPU


In [ ]:
net_one = net_one.to(device)

#### Choose the criterion, learning rate, and batch size.

In [ ]:
criterion = nn.CrossEntropyLoss()

lr = 0.5  # initial learning rate

bs = 10

#### Start training the 1-layer MLP Network

In [ ]:
start = time.time()

for epoch in range(100):
    if epoch % 10 == 0:
      lr = lr / 1.5

    optimizer=torch.optim.SGD(net_one.parameters(), lr=lr )
    
    running_loss=0
    running_error=0
    num_batches=0
    
    shuffled_indices=torch.randperm(train_size)
 
    for count in range(0, train_size, bs):
    
        optimizer.zero_grad()
        
        indices=shuffled_indices[count:count+bs]
        minibatch_data = train_data[indices]
        minibatch_label = train_label[indices]

        minibatch_data  = minibatch_data.to(device)
        minibatch_label = minibatch_label.to(device)

        if minibatch_data.size()[0] % bs == 0:
          inputs = minibatch_data.view(bs, image_size)
        else:
          inputs = minibatch_data.view(minibatch_data.size()[0], image_size)
        
        inputs = inputs.to(device)

        inputs.requires_grad_()

        scores=net_one( inputs ) 

        loss =  criterion( scores , minibatch_label) 
        
        loss.backward()

        optimizer.step()
        
        running_loss += loss.detach().item()
               
        error = get_error( scores.detach() , minibatch_label)
        running_error += error.item()
        
        num_batches+=1
    

    total_loss = running_loss/num_batches
    total_error = running_error/num_batches
    elapsed_time = time.time() - start
    print('epoch=',epoch, ' time=', elapsed_time,
          ' loss=', total_loss , ' error=', total_error*100, 'percent.')
    print(' ')

epoch= 0  time= 0.6481356620788574  loss= 9.638480291061107  error= 45.75221213267616 percent.
 
epoch= 1  time= 1.0489492416381836  loss= 8.988226090302378  error= 43.00884929730125 percent.
 
epoch= 2  time= 1.458594560623169  loss= 8.99622404126123  error= 43.51032418487346 percent.
 
epoch= 3  time= 1.8571250438690186  loss= 8.698874665898131  error= 42.03539780810871 percent.
 
epoch= 4  time= 2.259141445159912  loss= 8.512029104028901  error= 41.12094354137207 percent.
 
epoch= 5  time= 2.65352463722229  loss= 8.412483326475353  error= 41.00294951844005 percent.
 
epoch= 6  time= 3.0567638874053955  loss= 7.866935381033955  error= 40.11799369589769 percent.
 
epoch= 7  time= 3.4471328258514404  loss= 7.883326668127448  error= 40.560471431343956 percent.
 
epoch= 8  time= 3.862499952316284  loss= 7.845613889388636  error= 40.61946867138235 percent.
 
epoch= 9  time= 4.275045394897461  loss= 7.624699778140558  error= 39.46902600713184 percent.
 
epoch= 10  time= 4.678787708282471  

#### Evaluate on Test Set

In [ ]:
net_one.eval()
with torch.no_grad():
    data = test_data.view(test_size, image_size)
    labels = test_label
    scores = net_one(data) 
    print("Test error={} percent".format(get_error(scores, labels)*100))

Test error=40.31485366821289 percent


### Two Layer MLP

In [ ]:
class two_layer_net(nn.Module):

    def __init__(self, input_size, hidden_size1,  output_size):
        super(two_layer_net , self).__init__()

        self.layer1 = nn.Linear(  input_size   , hidden_size1 , bias=True  )
        self.layer2 = nn.Linear(  hidden_size1 , output_size  , bias=True  )
        
    def forward(self, x):
        
        y       = self.layer1(x)
        y_hat   = torch.relu(y)
        scores  = self.layer2(y_hat)
        
        return scores

In [ ]:
net_two = two_layer_net(image_size, 100, 2)
print(net_two)

two_layer_net(
  (layer1): Linear(in_features=784, out_features=100, bias=True)
  (layer2): Linear(in_features=100, out_features=2, bias=True)
)


#### Send the weights of the networks to the GPU


In [ ]:
net_two = net_two.to(device)

#### Choose the criterion, learning rate, and batch size.

In [ ]:
criterion = nn.CrossEntropyLoss()

lr = 0.5  # initial learning rate

bs = 10

#### Start training the 2-layer MLP Network

In [ ]:
start = time.time()

for epoch in range(100):
    if epoch % 10 == 0:
      lr = lr / 1.5

    optimizer=torch.optim.SGD(net_two.parameters(), lr=lr )
    
    running_loss=0
    running_error=0
    num_batches=0
    
    shuffled_indices=torch.randperm(train_size)
 
    for count in range(0, train_size, bs):
    
        optimizer.zero_grad()
        
        indices=shuffled_indices[count:count+bs]
        minibatch_data = train_data[indices]
        minibatch_label = train_label[indices]

        minibatch_data  = minibatch_data.to(device)
        minibatch_label = minibatch_label.to(device)

        if minibatch_data.size()[0] % bs == 0:
          inputs = minibatch_data.view(bs, image_size)
        else:
          inputs = minibatch_data.view(minibatch_data.size()[0], image_size)
        
        inputs = inputs.to(device)

        inputs.requires_grad_()

        scores=net_two( inputs ) 

        loss =  criterion( scores , minibatch_label) 
        
        loss.backward()

        optimizer.step()
        
        running_loss += loss.detach().item()
               
        error = get_error( scores.detach() , minibatch_label)
        running_error += error.item()
        
        num_batches+=1
    

    total_loss = running_loss/num_batches
    total_error = running_error/num_batches
    elapsed_time = time.time() - start
    print('epoch=',epoch, ' time=', elapsed_time,
          ' loss=', total_loss , ' error=', total_error*100, 'percent.')
    print(' ')

epoch= 0  time= 0.5544378757476807  loss= 0.6675033589995365  error= 36.814158926319585 percent.
 
epoch= 1  time= 1.053694725036621  loss= 0.6612104073738279  error= 36.90265431164992 percent.
 
epoch= 2  time= 1.5652492046356201  loss= 0.6574865317625985  error= 36.3126837925925 percent.
 
epoch= 3  time= 2.0581915378570557  loss= 0.657171582489942  error= 36.40117938891273 percent.
 
epoch= 4  time= 2.56384015083313  loss= 0.6613845460358623  error= 36.3421824126117 percent.
 
epoch= 5  time= 3.0845041275024414  loss= 0.6476951265229588  error= 36.13569263511703 percent.
 
epoch= 6  time= 3.598806619644165  loss= 0.6460965797964451  error= 36.34218234228174 percent.
 
epoch= 7  time= 4.109570741653442  loss= 0.6505577730394043  error= 36.75516166869869 percent.
 
epoch= 8  time= 4.600783109664917  loss= 0.6469511052148532  error= 36.165191307883696 percent.
 
epoch= 9  time= 5.1060028076171875  loss= 0.6397609703660363  error= 36.165191202388755 percent.
 
epoch= 10  time= 5.6359651

#### Evaluate on test set

In [ ]:
net_two.eval()
with torch.no_grad():
    data = test_data.view(test_size, image_size)
    labels = test_label
    scores = net_two(data) 
    print("Test error={} percent".format(get_error(scores, labels)*100))

Test error=33.67556381225586 percent


### Three Layer MLP

In [ ]:
class three_layer_net(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(three_layer_net , self).__init__()
        self.layer1 = nn.Linear(  input_size, hidden_size )
        self.layer2 = nn.Linear(  hidden_size, hidden_size )
        self.layer3 = nn.Linear(  hidden_size, output_size )
    def forward(self, x):
        x = self.layer1(x)
        x = torch.relu(x)
        x = self.layer2(x)
        x = torch.relu(x)
        score = self.layer3(x)
        return score

net_three = three_layer_net(image_size,25,10)
print(net_three)
net_three = net_three.to(device)

bs = 50
criterion = nn.CrossEntropyLoss()
optimizer=torch.optim.SGD( net_three.parameters() , lr=0.01 )

for epoch in range(100):
    running_loss=0
    running_error=0
    num_batches=0
    shuffled_indices=torch.randperm(train_size)
    for count in range(0,train_size,bs):
        optimizer.zero_grad()
        indices = shuffled_indices[count:count+bs]
        minibatch_data = train_data[indices]
        minibatch_label = train_label[indices]
        if minibatch_data.size()[0] % bs == 0:
          inputs = minibatch_data.view(bs, image_size)
        else:
          inputs = minibatch_data.view(minibatch_data.size()[0], image_size)
        inputs = inputs.to(device)
        inputs.requires_grad_()
        scores = net_three(inputs) 
        loss = criterion(scores, minibatch_label) 
        loss.backward()
        optimizer.step()
        running_loss += loss.detach().item()
        error = get_error( scores.detach() , minibatch_label)
        running_error += error.item()
        num_batches+=1
    total_loss = running_loss/num_batches
    total_error = running_error/num_batches
    print("Epoch={}, Train loss={}, Train error={}".format(epoch+1, total_loss, total_error))
    
net.eval()
with torch.no_grad():
    data = test_data.view(-1,image_size)
    labels = test_label
    scores = net_three(data) 
    print("Test error={} percent".format(get_error(scores, labels)*100))


three_layer_net(
  (layer1): Linear(in_features=784, out_features=25, bias=True)
  (layer2): Linear(in_features=25, out_features=25, bias=True)
  (layer3): Linear(in_features=25, out_features=10, bias=True)
)
Epoch=1, Train loss=1.5114822527941536, Train error=0.429089193834978
Epoch=2, Train loss=0.7215002407045925, Train error=0.3256262006128536
Epoch=3, Train loss=0.6825604728039574, Train error=0.3258064669721267
Epoch=4, Train loss=0.665534695281702, Train error=0.3262808515745051
Epoch=5, Train loss=0.6576124333283481, Train error=0.3266224072260015
Epoch=6, Train loss=0.6499484172638725, Train error=0.3230265834752251
Epoch=7, Train loss=0.6449912716360653, Train error=0.3237950793083976
Epoch=8, Train loss=0.6370415661264869, Train error=0.3221442287459093
Epoch=9, Train loss=0.6338594073758406, Train error=0.3248102621120565
Epoch=10, Train loss=0.6284342390649459, Train error=0.32200191126150246
Epoch=11, Train loss=0.6291309244492475, Train error=0.3203984963543275
Epoch=12,

### CNN Lenet 5

#### BW

In [31]:
class convnet_bw(nn.Module):
    def __init__(self):
        super(convnet_bw, self).__init__()
        self.conv1 = nn.Conv2d(1,   50,  kernel_size=3,  padding=1 )
        self.pool1  = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(50,  100,  kernel_size=3,  padding=1 )
        self.pool2 = nn.MaxPool2d(2, 2)
        self.linear1 = nn.Linear(4900, 100)
        self.linear2 = nn.Linear(100, 2)

    def forward(self, x):
        x = self.conv1(x)
        x = torch.relu(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = torch.relu(x)
        x = self.pool2(x)
        x = x.view(-1, 4900)
        x = self.linear1(x)
        x = torch.relu(x)
        x = self.linear2(x)
        return x

In [32]:
net=convnet_bw()
print(net)

convnet_bw(
  (conv1): Conv2d(1, 50, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(50, 100, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (linear1): Linear(in_features=4900, out_features=100, bias=True)
  (linear2): Linear(in_features=100, out_features=2, bias=True)
)


In [33]:
net = net.to(device)
mean = mean.to(device)
std = std.to(device)

In [34]:
criterion = nn.CrossEntropyLoss()
my_lr=0.25 
bs= 20

mean= train_data.mean()
print(mean)
std= train_data.std()
print(std)

tensor(0.4062, device='cuda:0')
tensor(0.2539, device='cuda:0')


In [35]:
start=time.time()

# for epoch in range(100):
for epoch in range(20):

    if not epoch % 10:
        my_lr = my_lr / 1.5
        
    optimizer=torch.optim.SGD( net.parameters() , lr=my_lr )
    
    running_loss=0
    running_error=0
    num_batches=0
    
    shuffled_indices=torch.randperm(train_size)
 
    for count in range(0, train_size, bs):
        
        # FORWARD AND BACKWARD PASS
    
        optimizer.zero_grad()
             
        indices=shuffled_indices[count:count+bs]
        minibatch_data =  train_data[indices].unsqueeze(dim=1)
        minibatch_label=  train_label[indices]
    
        minibatch_data =  minibatch_data.to(device)
        minibatch_label=  minibatch_label.to(device)
    
        inputs = (minibatch_data - mean)/std
        
        inputs.requires_grad_()

        scores=net( inputs ) 

        loss =  criterion( scores , minibatch_label) 
          
        loss.backward()
        
        optimizer.step()      
        
        running_loss += loss.detach().item()
        
        error = get_error( scores.detach() , minibatch_label)
        running_error += error.item()
        
        num_batches+=1        
    
    
    total_loss = running_loss/num_batches
    total_error = running_error/num_batches
    elapsed = (time.time()-start)/60
    
    print('epoch=',epoch, '\t time=', elapsed,'min', '\t lr=', my_lr  ,'\t loss=', total_loss , '\t error=', total_error*100 ,'percent')
    print(' ')

epoch= 0 	 time= 0.01554173231124878 min 	 lr= 0.16666666666666666 	 loss= 0.6223141934941797 	 error= 34.55882209188798 percent
 
epoch= 1 	 time= 0.027549342314402262 min 	 lr= 0.16666666666666666 	 loss= 0.6001183281926548 	 error= 32.470587211496685 percent
 
epoch= 2 	 time= 0.039292494455973305 min 	 lr= 0.16666666666666666 	 loss= 0.5864939544130774 	 error= 32.176469424191644 percent
 
epoch= 3 	 time= 0.051344807942708334 min 	 lr= 0.16666666666666666 	 loss= 0.5770802427740658 	 error= 30.88235167896046 percent
 
epoch= 4 	 time= 0.06310787200927734 min 	 lr= 0.16666666666666666 	 loss= 0.5536637199275634 	 error= 29.529410425354452 percent
 
epoch= 5 	 time= 0.07493263483047485 min 	 lr= 0.16666666666666666 	 loss= 0.5297401810393614 	 error= 27.647057561313403 percent
 
epoch= 6 	 time= 0.08667947848637898 min 	 lr= 0.16666666666666666 	 loss= 0.5024452802012949 	 error= 24.764704914654004 percent
 
epoch= 7 	 time= 0.098532768090566 min 	 lr= 0.16666666666666666 	 loss= 0.

In [38]:
net.eval()
with torch.no_grad():
    data = test_data.unsqueeze(dim=1)
    data = data.to(device)
    labels = test_label.to(device)
    scores = net(data) 
    print("Test error={} percent".format(get_error(scores, labels)*100))

Test error=34.97603988647461 percent


#### Color

### CNN VGG

#### BW

In [40]:
class convnet_VGG_bw(nn.Module):

    def __init__(self):

        super(convnet_VGG_bw, self).__init__()

        # block 1:         1 x 32 x 32 --> 64 x 16 x 16        
        self.conv1a = nn.Conv2d(1,   64,  kernel_size=3, padding=1 )
        self.conv1b = nn.Conv2d(64,  64,  kernel_size=3, padding=1 )
        self.pool1  = nn.MaxPool2d(2, 2)

        # block 2:         64 x 32 x 32 --> 128 x 16 x 16
        self.conv2a = nn.Conv2d(64,  128, kernel_size=3, padding=1 )
        self.conv2b = nn.Conv2d(128, 128, kernel_size=3, padding=1 )
        self.pool2  = nn.MaxPool2d(2, 2)

        # block 3:         128 x 8 x 8 --> 256 x 4 x 4        
        self.conv3a = nn.Conv2d(128, 256, kernel_size=3, padding=1 )
        self.conv3b = nn.Conv2d(256, 256, kernel_size=3, padding=1 )
        self.pool3  = nn.MaxPool2d(2, 2)
        
        #block 4:          256 x 4 x 4 --> 512 x 2 x 2
        self.conv4a = nn.Conv2d(256, 512, kernel_size=3, padding=1 )
        self.pool4  = nn.MaxPool2d(2, 2)

        # linear layers:   512 x 2 x 2 --> 2048 --> 4096 --> 4096 --> 10
        self.linear1 = nn.Linear(2048, 4096)
        self.linear1 = nn.Linear(2048, 4096)
        self.linear2 = nn.Linear(4096, 4096)
        self.linear3 = nn.Linear(4096, 2)


    def forward(self, x):

        # block 1:         1 x 32 x 32 --> 64 x 16 x 16
        x = self.conv1a(x)
        x = torch.relu(x)
        x = self.conv1b(x)
        x = torch.relu(x)
        x = self.pool1(x)
        
        # block 2:         64 x 16 x 16 --> 128 x 8 x 8
        x = self.conv2a(x)
        x = torch.relu(x)
        x = self.conv2b(x)
        x = torch.relu(x)
        x = self.pool2(x)
        
        # block 3:         128 x 8 x 8 --> 256 x 4 x 4
        x = self.conv3a(x)
        x = torch.relu(x)
        x = self.conv3b(x)
        x = torch.relu(x)
        x = self.pool3(x)
        
        #block 4:          256 x 4 x 4 --> 512 x 2 x 2
        x = self.conv4a(x)
        x = torch.relu(x)
        x = self.pool4(x)
        
        # linear layers:   512 x 2 x 2 --> 2048 --> 4096 --> 4096 --> 10
        x = x.view(-1, 2048)
        x = self.linear1(x)
        x = torch.relu(x)
        x = self.linear2(x)
        x = torch.relu(x)
        x = self.linear3(x) 
        
        return x

In [41]:
net=convnet_VGG_bw()
print(net)

convnet_VGG_bw(
  (conv1a): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv1b): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2a): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2b): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3a): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3b): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv4a): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (linear1): Linear(in_features=2048, out_features=4096, bias=True)
  (linear2): Linea

In [42]:
net = net.to(device)
mean = mean.to(device)
std = std.to(device)

In [43]:
criterion = nn.CrossEntropyLoss()
my_lr = 0.25 
bs = 20

In [44]:
start=time.time()

# for epoch in range(100):
for epoch in range(20):

    if not epoch % 10:
        my_lr = my_lr / 1.5
        
    optimizer=torch.optim.SGD( net.parameters() , lr=my_lr )
    
    running_loss=0
    running_error=0
    num_batches=0
    
    shuffled_indices=torch.randperm(train_size)
 
    for count in range(0, train_size, bs):
        
        # FORWARD AND BACKWARD PASS
    
        optimizer.zero_grad()
             
        indices=shuffled_indices[count:count+bs]
        minibatch_data =  train_data[indices].unsqueeze(dim=1)
        minibatch_label=  train_label[indices]
    
        minibatch_data =  minibatch_data.to(device)
        minibatch_label=  minibatch_label.to(device)
        
        inputs = (minibatch_data - mean)/std      # ONLY CHANGE IS HERE!
        
        inputs.requires_grad_()

        scores=net( inputs ) 

        loss =  criterion( scores , minibatch_label) 
          
        loss.backward()
        
        optimizer.step()      
        
        running_loss += loss.detach().item()
        
        error = get_error( scores.detach() , minibatch_label)
        running_error += error.item()
        
        num_batches+=1        
    
    
    total_loss = running_loss/num_batches
    total_error = running_error/num_batches
    elapsed = (time.time()-start)/60
    
    print('epoch=',epoch, '\t time=', elapsed,'min', '\t lr=', my_lr  ,'\t loss=', total_loss , '\t error=', total_error*100 ,'percent')
    print(' ')
    

RuntimeError: ignored

In [ ]:
net.eval()
with torch.no_grad():
    data = test_data_bw.unsqueeze(dim=1)
    data = data.to(device)
    labels = test_label_bw.to(device)
    scores = net(data) 
    print("Test error={} percent".format(get_error(scores, labels)*100))

#### Color